In [8]:
from random import choice

from game import valid, win_condition, next_player, game_updater, get_optimal, Board


initial_board, initial_turn, initial_player, initial_winning_condition = game_updater(
    moves=[]
)

likelihood_board: Board = [row[:] for row in initial_board]
initial_moves = valid(initial_board)

for initial_move in initial_moves:
    n = 100
    x = 0
    o = 0
    tie = 0
    turns = []

    for m in range(n):
        board = [row[:] for row in initial_board]
        turn = initial_turn
        player = initial_player
        winning_condition = initial_winning_condition
        while not winning_condition:
            opponent = "O" if player == "X" else "X"
            valid_moves = valid(board)
            winning_moves = get_optimal(board, valid_moves, player)
            blocking_moves = get_optimal(board, valid_moves, opponent)
            if len(winning_moves):
                valid_moves = winning_moves
            if len(blocking_moves):
                valid_moves = blocking_moves
            if initial_move and turn == initial_turn:
                valid_moves = [initial_move]
            if len(valid_moves):
                i, j = choice(valid_moves)
                board[i][j] = player
                winning_condition = win_condition(board, player)
                if winning_condition:
                    if player == "X":
                        x += 1
                    if player == "O":
                        o += 1
                    break
                else:
                    player = next_player(player)
                    if player == "X":
                        turn += 1
            else:
                tie += 1
                winning_condition = "tie"
        turns.append(turn)
    u, v = initial_move
    likelihood_board[u][v] = (x, o, tie)

In [9]:
from math import sqrt


def get_stats(outcome):
    x, o, tie = outcome
    total = x + o + tie
    x_prob = x / total
    o_prob = o / total
    tie_prob = tie / total
    median = x_prob - o_prob
    variance = (
        ((0 - median) ** 2 * tie_prob)
        + ((1 - median) ** 2 * x_prob)
        + ((-1 - median) ** 2 * o_prob)
    )
    std_dev = sqrt(variance)
    return (round(median, 3), round(std_dev, 3))


result = [
    [get_stats(item) if isinstance(item, tuple) else item for item in row]
    for row in likelihood_board
]

for row in result:
    print(row)

[(0.1, 0.975), None, None, None, None, None, (0.13, 0.956)]
[(-0.01, 0.975), None, None, None, None, None, (0.06, 0.978)]
[(-0.01, 0.964), None, None, None, None, None, (-0.02, 0.98)]
[(-0.23, 0.947), None, None, None, None, None, (-0.1, 0.975)]
[(-0.15, 0.984), None, None, None, None, None, (-0.01, 0.964)]
[(-0.19, 0.956), None, None, None, None, None, (-0.14, 0.959)]
[(0.02, 0.98), None, None, None, None, None, (-0.11, 0.958)]
